In [5]:
from transformers import pipeline
import sys
import random


# 모델 불러오기
sentiment_model = pipeline("sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")
intent_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
gen_model = pipeline("text-generation", model="skt/kogpt2-base-v2", tokenizer="skt/kogpt2-base-v2")

# 사용자에게 다마고치 이름 받기
name = input("🐣 당신의 다마고치 이름을 지어주세요: ")

# 초기 상태
pet = {
    "name": name,
    "age": 0,
    "stage": "유아기",
    "hunger": 20,
    "energy": 80,
    "cleanliness": 80,
    "affection": 50,
    "mood": "좋음 ❣️",
    "evolved": False,
    "alive": True
}

# 상태 출력
def show_status():
    print(f"\n🧸 {pet['name']} 상태")
    print(f"나이: {pet['age']}세 ({pet['stage']})")
    print(f"기분: {pet['mood']}")
    print(f"배고픔: {pet['hunger']}/100")
    print(f"에너지: {pet['energy']}/100")
    print(f"청결도: {pet['cleanliness']}/100")
    print(f"친밀도: {pet['affection']}/100")

# 턴 진행
def next_turn():
    pet["age"] += 1
    pet["hunger"] = min(100, pet["hunger"] + 10)
    pet["energy"] = max(0, pet["energy"] - 10)
    pet["cleanliness"] = max(0, pet["cleanliness"] - 10)
    update_stage()
    trigger_event()

# 성장 단계 업데이트
def update_stage():
    age = pet["age"]
    if age >= 10 and not pet["evolved"]:
        pet["stage"] = "성장기"
    if age >= 20 and not pet["evolved"] and pet["affection"] >= 80:
        pet["stage"] = "성체"
        pet["evolved"] = True
        print(f"\n🌟 {pet['name']}이(가) 성체로 성장했어요! 🐉")

        print(f"\n🎉 {pet['name']}은(는) 완전히 성장했습니다! 🎊")
        choice = input("새로운 다마고치를 키울까요? (네 / 아니오) → ")
        if choice.strip() == "네":
            reset_pet()
            print(f"\n🎮 새로운 다마고치 '{pet['name']}' 키우기 시작!")
        else:
            print("👋 게임을 종료합니다. 다음에 또 만나요!")
            sys.exit()

# 🎉 이벤트 발생 함수
def trigger_event():
    age = pet["age"]
    triggered = False

    # 유아기 이벤트 (3턴)
    if age == 3:
        event = random.choice([
            {"desc": "갑자기 토를 했어요 🤢 청결도 감소!", "cleanliness": -20},
            {"desc": "밤에 울었어요 😢 에너지 감소!", "energy": -20},
            {"desc": "장난감 발견! 🎁 기분 좋아졌어요!", "mood": "기분 좋아요 😊", "affection": +10}
        ])
        triggered = True

    # 추가된 중간 이벤트 (8턴)
    elif age == 8:
      event = random.choice([
          {"desc": "밤새 꿈을 꿨어요 🌙 기분이 이상해요", "mood": "우울해요 😢", "affection": -5},
          {"desc": "간식을 나눠줬어요 🍬 친밀도 증가!", "affection": +10},
          {"desc": "갑자기 비가 왔어요 🌧️ 청결도 감소", "cleanliness": -10}
      ])
      triggered = True

    # 성장기 이벤트 (13턴)
    elif age == 13:
        event = random.choice([
            {"desc": "과식했어요! 배가 너무 불러요 🍽️", "hunger": +20},
            {"desc": "목욕을 싫어했어요 🧼 청결도 감소!", "cleanliness": -15, "affection": -5},
            {"desc": "산책 중 다쳤어요 🩹 에너지 감소", "energy": -30}
        ])
        triggered = True

    # 성체 직전 이벤트 (19턴)
    elif age == 19:
        event = random.choice([
            {"desc": "질투심 폭발 😤 기분 나빠졌어요", "mood": "우울해요 😢", "affection": -10},
            {"desc": "깜짝 선물 🎁 친밀도 급상승!", "affection": +20},
            {"desc": "기절할 뻔! 😵 에너지 급감", "energy": -40}
        ])
        triggered = True

    if triggered:
        print(f"\n📢 [이벤트 발생] {event['desc']}")
        pet["hunger"] = min(100, pet["hunger"] + event.get("hunger", 0))
        pet["energy"] = max(0, pet["energy"] + event.get("energy", 0))
        pet["cleanliness"] = max(0, pet["cleanliness"] + event.get("cleanliness", 0))
        pet["affection"] = max(0, min(100, pet["affection"] + event.get("affection", 0)))
        if "mood" in event:
            pet["mood"] = event["mood"]


# 죽음 체크
def check_death():
    if pet["hunger"] >= 100 or pet["energy"] <= 0 or pet["cleanliness"] <= 0:
        pet["alive"] = False
        print(f"\n💀 {pet['name']}가 건강을 잃고 무지개 다리를 건넜어요... R.I.P 🕊️")
        return True
    return False

# 상태 질문 확인
def is_status_question(cmd):
    if any(kw in cmd for kw in ["배고", "먹을래", "밥 줄까"]):
        return f"{pet['name']}: {'너무 배고파요! 밥 줘!!! 🍚' if pet['hunger'] > 60 else '지금은 괜찮아요~ 😊'}"
    elif any(kw in cmd for kw in ["씻겨", "더러워", "씻어줄까"]):
        return f"{pet['name']}: {'좀 더럽긴 해요... 씻겨줘요! 🧼' if pet['cleanliness'] < 40 else '나 깨끗해요~ ✨'}"
    elif any(kw in cmd for kw in ["졸려", "자고", "잠"]):
        return f"{pet['name']}: {'너무 졸려요... 자고 싶어요 💤' if pet['energy'] < 30 else '아직은 괜찮아요! 🔋'}"
    elif any(kw in cmd for kw in ["기분", "우울", "행복", "기뻐"]):
        return f"{pet['name']}: 지금 기분은 {pet['mood']} 이에요!"
    return None

# 행동 수행
def perform_action(cmd):
    print(f"👤 당신: {cmd}")

    # 상태 질문인 경우 응답만
    status_reply = is_status_question(cmd)
    if status_reply:
        print(f"🗨️ {status_reply}")
        return

    # 정확한 명령에 대해서만 행동 실행
    if cmd == "밥 주기":
        pet["hunger"] = max(0, pet["hunger"] - 30)
        pet["affection"] = min(100, pet["affection"] + 5)
        prompt = f"{pet['name']}: 밥 너무 맛있다! 고마워요!"
    elif cmd == "씻기기":
        pet["cleanliness"] = min(100, pet["cleanliness"] + 30)
        prompt = f"{pet['name']}: 깨끗해졌어요! 최고!"
    elif cmd == "재우기":
        pet["energy"] = min(100, pet["energy"] + 40)
        prompt = f"{pet['name']}: 이제 좀 살 것 같아요~"
    else:  # 자연어 대화
        result = sentiment_model(cmd)[0]
        if result["label"] == "positive":
            pet["mood"] = "기분 좋아요 😊"
            pet["affection"] = min(100, pet["affection"] + 10)
            prompt = f"{pet['name']}: 너무 좋아요~!"
        else:
            pet["mood"] = "우울해요 😢"
            pet["affection"] = max(0, pet["affection"] - 5)
            prompt = f"{pet['name']}: 기분이 좀 안 좋아요..."

    # 말 생성
    response = gen_model(prompt, max_new_tokens=20, do_sample=True, truncation=True,
                         pad_token_id=gen_model.tokenizer.eos_token_id)[0]["generated_text"]
    print(f"🗨️ {response.strip()}")

# 다마고치 리셋
def reset_pet():
    global pet
    name = input("새로운 다마고치 이름을 지어주세요: ")
    pet = {
        "name": name,
        "age": 0,
        "stage": "유아기",
        "hunger": 20,
        "energy": 80,
        "cleanliness": 80,
        "affection": 50,
        "mood": "좋음 ❣️",
        "evolved": False,
        "alive": True
    }

# 📌 사용법 안내
def show_guide():
    print("""
📘 다마고치 키우기 메뉴얼

🎮 당신은 귀여운 다마고치를 키우는 보호자입니다!
다마고치의 상태는 매 턴마다 변화하며, 올바른 관리를 통해 건강하게 키워야 합니다. 💪

👉 목표는 다마고치를 건강하고 행복하게 키워서 성체로 진화시키는 것! 🐣
다마고치가 성체로 진화하면 게임은 종료됩니다! ('종료'라고 입력 시 게임은 즉시 종료됩니다.)

──────────────────────────────
🛠️ [1] 행동 명령어 사용법

👉 정확한 명령어를 입력하면 실제로 상태가 바뀝니다:

  ✅ 밥 주기     → 배고픔 감소, 친밀도 +5
  ✅ 씻기기     → 청결도 증가
  ✅ 재우기     → 에너지 회복

※ 위 3가지 명령어로만 상태가 실제로 바뀝니다!
──────────────────────────────
💬 [2] 자유로운 대화도 가능해요!

다마고치에게 말을 걸어보세요! 예를 들어:

  - 배고파?
  - 씻겨줄까?
  - 졸려?
  - 기분 어때?
  - 나 좋아해?

→ 다마고치는 현재 상태에 따라 감정을 표현해줘요 😊

──────────────────────────────
⚠️ [3] 다마고치가 죽는 조건

❌ 아래 조건 중 하나라도 만족하면 다마고치는 무지개 다리를 건너요... 😢

  - 배고픔 ≥ 100
  - 에너지 ≤ 0
  - 청결도 ≤ 0

💡 자주 밥도 주고, 씻기고, 재워서 건강하게 키워주세요!
──────────────────────────────
💖 [4] 친밀도 & 기분 변화

- 기분 좋은 말을 하면 → 기분 👍 + 친밀도 +10
- 기분 나쁜 말을 하면 → 기분 👎 + 친밀도 -5
- 밥 주면 친밀도 +5

🤝 친밀도가 높아지면 다마고치는 더 빠르게 진화합니다!
──────────────────────────────
⚡ [5] 에너지 변화

- 매 턴마다 에너지가 10씩 감소해요
- ‘재우기’ 명령어로 +40 회복 가능해요

🛌 너무 피곤해지기 전에 재워주세요!
──────────────────────────────

🌱 [6] 성장과 진화

- 다마고치가 일정 턴 이상 자라고
- 친밀도가 높아지면

👉 귀여운 모습에서 성체로 진화합니다! 🐲
진화 후에는 게임을 계속할지, 새로운 다마고치를 키울지 선택할 수 있어요!
──────────────────────────────

🌈 [7] 랜덤 성장 이벤트

- 특정 나이에 도달하면 예기치 못한 이벤트가 발생할 수 있어요!

   * 예: 감기, 과식, 기분 변화 등

- 이벤트는 다마고치에게 좋은 영향을 줄 수도, 나쁜 영향을 줄 수도 있어요 😨

→ 항상 상태를 잘 관리하고 대비하세요!
──────────────────────────────

🎉  이제 다마고치에게 말을 걸어보세요!

예:
👉 “안녕!”
👉 “기분 어때?”
👉 “밥 주기”

당신의 따뜻한 말 한마디가 다마고치에게 큰 힘이 돼요 😊

""")

# 게임 루프
show_guide()
print(f"\n🎮 다마고치 '{pet['name']}' 키우기 시작!")

while True:
    next_turn()
    show_status()
    cmd = input("\n📝 입력하세요 → ").strip()

    if cmd in ["종료", "exit", "그만"]:
        print("👋 게임을 종료합니다. 다음에 또 만나요!")
        break

    perform_action(cmd)


Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


🐣 당신의 다마고치 이름을 지어주세요: 종료

📘 다마고치 키우기 메뉴얼

🎮 당신은 귀여운 다마고치를 키우는 보호자입니다!
다마고치의 상태는 매 턴마다 변화하며, 올바른 관리를 통해 건강하게 키워야 합니다. 💪

👉 목표는 다마고치를 건강하고 행복하게 키워서 성체로 진화시키는 것! 🐣
다마고치가 성체로 진화하면 게임은 종료됩니다! ('종료'라고 입력 시 게임은 즉시 종료됩니다.)

──────────────────────────────
🛠️ [1] 행동 명령어 사용법

👉 정확한 명령어를 입력하면 실제로 상태가 바뀝니다:

  ✅ 밥 주기     → 배고픔 감소, 친밀도 +5
  ✅ 씻기기     → 청결도 증가
  ✅ 재우기     → 에너지 회복

※ 위 3가지 명령어로만 상태가 실제로 바뀝니다!
──────────────────────────────
💬 [2] 자유로운 대화도 가능해요!

다마고치에게 말을 걸어보세요! 예를 들어:

  - 배고파?
  - 씻겨줄까?
  - 졸려?
  - 기분 어때?
  - 나 좋아해?

→ 다마고치는 현재 상태에 따라 감정을 표현해줘요 😊

──────────────────────────────
⚠️ [3] 다마고치가 죽는 조건

❌ 아래 조건 중 하나라도 만족하면 다마고치는 무지개 다리를 건너요... 😢

  - 배고픔 ≥ 100
  - 에너지 ≤ 0
  - 청결도 ≤ 0

💡 자주 밥도 주고, 씻기고, 재워서 건강하게 키워주세요!
──────────────────────────────
💖 [4] 친밀도 & 기분 변화

- 기분 좋은 말을 하면 → 기분 👍 + 친밀도 +10
- 기분 나쁜 말을 하면 → 기분 👎 + 친밀도 -5
- 밥 주면 친밀도 +5

🤝 친밀도가 높아지면 다마고치는 더 빠르게 진화합니다!
──────────────────────────────
⚡ [5] 에너지 변화

- 매 턴마다 에너지가 10씩 감소해요
- ‘재우기’ 명령어로 +40 회복 가능해요

🛌 너무 피곤해지기 전에 재워주세요!
─────

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


👤 당신: 정려
🗨️ 종료: 너무 좋아요~!!!
#여자들의날 #여자들의날 #여자들의날 #여자들의

🧸 종료 상태
나이: 2세 (유아기)
기분: 기분 좋아요 😊
배고픔: 40/100
에너지: 60/100
청결도: 60/100
친밀도: 60/100

📝 입력하세요 → 종료
👋 게임을 종료합니다. 다음에 또 만나요!
